<a href="https://colab.research.google.com/github/owahltinez/object-detection-utils/blob/main/object_detection_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Download prebuilt object_detection library binaries and data
!wget -c -q https://github.com/owahltinez/object-detection-utils/releases/download/462bb852d/object_detection-0.1-py3-none-any.whl
!wget -c -q https://github.com/owahltinez/object-detection-utils/releases/download/462bb852d/ducky.zip && rm -rf ducky && unzip -q -d ducky ducky.zip
!pip install object_detection-0.1-py3-none-any.whl > /dev/null
!rm -rf object_detection-0.1-py3-none-any.whl ducky.zip

In [ ]:
#@title Download object-detection-utils libraries
!git clone --depth 1 https://github.com/owahltinez/object-detection-utils
!mv object-detection-utils/* .
!rm -rf object-detection-utils

In [ ]:
#@title Imports and utility functions
import os
import tensorflow as tf
import numpy as np

from PIL import Image
from object_detection.utils import visualization_utils as viz_utils

# These are the object-detection-utils library imports.
import dataset_util
import keras_model_wrapper
import pretrained_models

def plot_detections(
    image_np,
    boxes,
    classes,
    scores,
    category_index,
):
  annotated_image = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      annotated_image,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.5)
  display(Image.fromarray(annotated_image))

In [ ]:
#@title Load the pretrained detection model
detection_model = pretrained_models.load_detection_model('ssd_resnet50_v1_fpn_640x640', num_classes=1)

In [ ]:
#@title Convert it into a keras compatible model
keras_model = keras_model_wrapper.KerasModelWrapper(detection_model)

In [ ]:
#@title Create train image dataset
train_image_dir = 'ducky/train/'
image_paths = [os.path.join(train_image_dir, f) for f in sorted(os.listdir(train_image_dir))]
detection_boxes = [
    [[0.436, 0.591, 0.629, 0.712]],
    [[0.539, 0.583, 0.73, 0.71]],
    [[0.464, 0.414, 0.626, 0.548]],
    [[0.313, 0.308, 0.648, 0.526]],
    [[0.256, 0.444, 0.484, 0.629]],
]
detection_classes = [[1]] * len(image_paths)

ds = dataset_util.image_dataset_from_paths(
    image_paths,
    detection_boxes,
    detection_classes,
    batch_size=5,
)

In [ ]:
#@title Fine-tune the model using the training data

# Use specific variables for fine-tuning and keep the rest frozen.
prefixes_to_train = [
    "WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead",
    "WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead",
]
trainable_variables = []
for model_variable in detection_model.trainable_variables:
    if any([model_variable.name.startswith(prefix) for prefix in prefixes_to_train]):
        trainable_variables.append(model_variable)
keras_model.set_trainable_variables(trainable_variables)

keras_model.compile(optimizer=tf.keras.optimizers.Adam(), run_eagerly=True)
keras_model.fit(ds.unbatch().cache().shuffle(100).batch(5), epochs=100);

In [ ]:
#@title Run inference using unseen test data

category_index = {1: {'id': 1, 'name': 'rubber_ducky'}}
test_image_dir = 'ducky/test/'
image_paths = [os.path.join(test_image_dir, f) for f in sorted(os.listdir(test_image_dir))]
image_arrays = [dataset_util.read_image_tensor(p) for p in image_paths]

for image in image_arrays:
  pred = keras_model.predict(tf.expand_dims(image, axis=0), verbose=0)
  # Retrieve only the first prediction, since it's a single-item batch.
  pred = {k: v[0] for k, v in pred.items()}
  plot_detections(
    image.numpy().astype(np.uint8),
    pred['detection_boxes'],
    pred['detection_classes'],
    pred['detection_scores'],
    category_index,
  )